<a href="https://colab.research.google.com/github/erictong0/FrederickLab2023/blob/main/Linear%20Regression/Initial%20defining%20regions/230619_OLD_final_combined_linear_regression_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bio --upgrade
!pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.9/661.9 kB 10.0 MB/s eta 0:00:00


In [ ]:
import sys
import glob
import math
import argparse
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as pltticker
import csv
from matplotlib import colors
import re
import pandas as pd
from numpy import savetxt
from sklearn.linear_model import LinearRegression
from statistics import mean
import statsmodels.api as sm
from Bio.Data.IUPACData import protein_letters_1to3 as one2three
import scipy.optimize as spo

In [ ]:
fwhm = 1.5  # Full width at half maximum
# Calculate the standard deviation from the FWHM
c = fwhm / (2 * np.sqrt(2 * np.log(2)))

def get_row_floats(data, row):
  return list(map(lambda x: float(x[row]), data))

xArray = []
for i in range(68):
  b = i / 4 + 54
  xArray.append(np.linspace(b - 3 * c, b + 3 * c, 100))

oData = []
data = []
with open("T_10K.out", mode="r") as infile: #open data file and read contents
  reader = csv.reader(infile, delimiter="\n")
  for row in reader:
    if len(row) == 1:
      data.append(re.split('\t', row[0])[2:])
    else:
      data.append(row)

tempData = []
for i in range(6):
  tempData.append(get_row_floats(data[1:], i))

oData.append(tempData)

data = []
tempData = []
count = 0
with open("nanodisc.csv", mode="r") as infile: #open data file and read contents
  chunk_size = 1000
  while True:
      chunk = infile.readlines(chunk_size)
      if not chunk:
        break
      for line in chunk:
        reader = csv.reader(infile)
        for row in reader:
          count += 1
          if len(row) == 1:
            tempData.append(re.split(r',', row[0]))
          else:
            tempData.append(row)

data.append(get_row_floats(tempData,0))
data.append(get_row_floats(tempData,2))

oData.append(data)

DATAINPUT = oData

In [ ]:
def main(inData, aa, rangeChoices):
  aa1 = aa

  phi = inData[0][0]
  psi = inData[0][1]
  ca = inData[0][2]
  cb = inData[0][3]
  c = inData[0][4]
  n = inData[0][5]



  ca_exp = inData[1][0] #def x as Ca shift
  intensity = inData[1][1]



  atom_key={
            "ca":0,
            "cb":1,
            "co":2,
            "n":3,
          }

  nonCys_nonGly_aas={"A", "D", "E", "F", "H", "I", "K", "L", "M",
                    "N", "P", "Q", "R", "S", "T", "V", "Y", "W"}

  nonCys_aas={"A", "D", "E", "F", "G", "H", "I", "K", "L", "M",
              "N", "P", "Q", "R", "S", "T", "V", "Y", "W"}

  na=np.nan

  aa_spec_ranges={ #ranges of interest for chemical shifts
                  #aa1 Ca      Cb      C         N
                  # Ca & Cb adjusted to 17ppm to match Fig5
                 'A':[[11,28],[203,234],[170,184],[112,133]],
                  'R':[[47,64],[22,39],[169,183],[112,133]],
                  'N':[[44,61],[30,47],[168,182],[106,127]],
                  'D':[[45,62],[33,50],[169,183],[109,130]],
  #               'C':[[50,67],[22,39],[168,182],[105,126]],
                  'Q':[[47,64],[22,39],[169,183],[108,129]],
                  'E':[[49,66],[22,39],[169,183],[109,130]],
                  'G':[[36,53],[203,234],[167,181],[ 98,119]],
                  'H':[[49,66],[23,40],[168,182],[107,128]],
                  'I':[[54,71],[31,48],[168,182],[109,130]],
                  'L':[[45,62],[33,50],[170,184],[110,131]],
                  'K':[[49,66],[26,43],[169,183],[109,130]],
                  'M':[[47,64],[26,43],[169,183],[108,129]],
                  'F':[[50,67],[31,48],[168,182],[107,128]],
                  'P':[[57,74],[25,42],[170,184],[ na, na]],
                  'S':[[50,67],[105,139],[55,72],[167,181],[104,125]],
                  'T':[[54,71],[104,138],[60,77],[168,182],[102,123]],
                  'W':[[48,65],[21,38],[169,183],[109,130]],
                  'Y':[[50,67],[30,47],[168,182],[108,129]],
                  'V':[[54,71],[25,42],[169,183],[108,129]],
                  }

  ####Changed this part####
  rchoice = rangeChoices

  ### extract plot subregions ###
  # "r" stands for ramachandran
  # regions: beta-strand, polyproline helix ii
  #          beta-turn I, alpha-helix,
  #          left handed helix (all phi > 0)
  # !!! top and left corner are always >= or <=
  #     unless they reach the bottom/right axis
  #     (i.e. betaB, ppiiB, left)

  #filter beta regions
  mask1 = np.logical_and(np.array(phi) >= rchoice['rbetaA'][0], np.array(phi) <= rchoice['rbetaA'][1])
  mask1 = np.logical_and(mask1, np.array(psi) >= rchoice['rbetaA'][2])
  mask1 = np.logical_and(mask1, np.array(psi) <= rchoice['rbetaA'][3])

  mask2 = np.logical_and(np.array(phi) >= rchoice['rbetaB'][0], np.array(phi) <= rchoice['rbetaB'][1])
  mask2 = np.logical_and(mask1, np.array(psi) >= rchoice['rbetaB'][2])
  mask2 = np.logical_and(mask1, np.array(psi) <= rchoice['rbetaB'][3])

  mask = np.logical_or(mask1, mask2)

  data_beta_ca = np.array(ca)[mask]
  data_beta_cb = np.array(cb)[mask]
  data_beta_c = np.array(c)[mask]
  data_beta_n = np.array(n)[mask]

  #filter ppii regions
  mask1 = np.logical_and(np.array(phi) >= rchoice['rppiiA'][0], np.array(phi) <= rchoice['rppiiA'][1])
  mask1 = np.logical_and(mask1, np.array(psi) >= rchoice['rppiiA'][2])
  mask1 = np.logical_and(mask1, np.array(psi) <= rchoice['rppiiA'][3])

  mask2 = np.logical_and(np.array(phi) >= rchoice['rppiiB'][0], np.array(phi) <= rchoice['rppiiB'][1])
  mask2 = np.logical_and(mask1, np.array(psi) >= rchoice['rppiiB'][2])
  mask2 = np.logical_and(mask1, np.array(psi) <= rchoice['rppiiB'][3])

  mask = np.logical_or(mask1, mask2)

  data_ppii_ca = np.array(ca)[mask]
  data_ppii_cb = np.array(cb)[mask]
  data_ppii_c = np.array(c)[mask]
  data_ppii_n = np.array(n)[mask]

  #filter ddgg
  mask = np.logical_and(np.array(phi) >= rchoice['rdelta'][0], np.array(phi) <= rchoice['rdelta'][1])
  mask = np.logical_and(mask1, np.array(psi) >= rchoice['rdelta'][2])
  mask = np.logical_and(mask1, np.array(psi) <= rchoice['rdelta'][3])

  data_ddgg_ca = np.array(ca)[mask]
  data_ddgg_cb = np.array(cb)[mask]
  data_ddgg_c = np.array(c)[mask]
  data_ddgg_n = np.array(n)[mask]

  #filter turn
  mask1 = np.logical_and(np.array(phi) >= rchoice['rturnI'][0], np.array(phi) < rchoice['ralpha'][0])
  mask1 = np.logical_and(mask1, np.array(psi) >= rchoice['rturnI'][2])
  mask1 = np.logical_and(mask1, np.array(psi) <= rchoice['rturnI'][3])

  mask2 = np.logical_and(np.array(phi) >= rchoice['ralpha'][1], np.array(phi) < rchoice['rturnI'][1])
  mask2 = np.logical_and(mask1, np.array(psi) >= rchoice['rturnI'][2])
  mask2 = np.logical_and(mask1, np.array(psi) <= rchoice['rturnI'][3])

  mask3 = np.logical_and(np.array(phi) >= rchoice['ralpha'][0], np.array(phi) < rchoice['ralpha'][1])
  mask3 = np.logical_and(mask1, np.array(psi) >= rchoice['rturnI'][2])
  mask3 = np.logical_and(mask1, np.array(psi) <= rchoice['ralpha'][2])

  mask4 = np.logical_and(np.array(phi) >= rchoice['ralpha'][0], np.array(phi) < rchoice['ralpha'][1])
  mask4 = np.logical_and(mask1, np.array(psi) > rchoice['ralpha'][3])
  mask4 = np.logical_and(mask1, np.array(psi) <= rchoice['rturnI'][3])

  mask = np.logical_or(mask1, np.logical_or(mask2, np.logical_or(mask3, mask4)))
  data_turn_ca = np.array(ca)[mask]
  data_turn_cb = np.array(cb)[mask]
  data_turn_c = np.array(c)[mask]
  data_turn_n = np.array(n)[mask]

  #filter alph
  mask = np.logical_and(np.array(phi) >= rchoice['ralpha'][0], np.array(phi) < rchoice['ralpha'][1])
  mask = np.logical_and(mask1, np.array(psi) > rchoice['ralpha'][2])
  mask = np.logical_and(mask1, np.array(psi) <= rchoice['ralpha'][3])

  data_alph_ca = np.array(ca)[mask]
  data_alph_cb = np.array(cb)[mask]
  data_alph_c = np.array(c)[mask]
  data_alph_n = np.array(n)[mask]

  #filter left
  mask = np.logical_and(np.array(phi) >= rchoice['rlhelx'][0], np.array(phi) <= rchoice['rlhelx'][1])
  mask = np.logical_and(mask1, np.array(psi) >= rchoice['rlhelx'][2])
  mask = np.logical_and(mask1, np.array(psi) <= rchoice['rlhelx'][3])

  data_left_ca = np.array(ca)[mask]
  data_left_cb = np.array(cb)[mask]
  data_left_c = np.array(c)[mask]
  data_left_n = np.array(n)[mask]


  # create dictionaries of data sets

  cs_ca = ({'beta': data_beta_ca,
            'ppii': data_ppii_ca,
            'ddgg': data_ddgg_ca,
            'left': data_left_ca,
            'turn': data_turn_ca,
            'alph': data_alph_ca,
            })
  cs_cb = ({'beta': data_beta_cb,
            'ppii': data_ppii_cb,
            'ddgg': data_ddgg_cb,
            'left': data_left_cb,
            'turn': data_turn_cb,
            'alph': data_alph_cb,
            })
  cs_co = ({'beta': data_beta_c,
            'ppii': data_ppii_c,
            'ddgg': data_ddgg_c,
            'left': data_left_c,
            'turn': data_turn_c,
            'alph': data_alph_c,
            })

  cs_all = {'ca':cs_ca,
            'cb':cs_cb,
            'c' :cs_co,
            }

  bin_width=0.25
  mtick_spacing=5
  #fig, axs = plt.subplots(1, 1, sharey=False, tight_layout=True,figsize=(10,10)) #35826579

  x_lims = np.array(aa_spec_ranges[aa1][0])
  y_lims = np.array(aa_spec_ranges[aa1][1])
  x_edges = np.arange(x_lims[0],x_lims[1]+bin_width,bin_width) #create an array from one limit to the next using bin widths
  y_edges=np.arange(y_lims[0],y_lims[1]+bin_width,bin_width)
  hw=bin_width/2
  center_bins_x=np.arange(x_lims[0]+hw,x_lims[1]+hw,bin_width)
  center_bins_y=np.arange(y_lims[0]+hw,y_lims[1]+hw,bin_width)
  inputs = []
  h_lists = []
  h_list = []
  c = fwhm / (2 * np.sqrt(2 * np.log(2)))

  for key in cs_ca:
    H, bins = np.histogram(cs_ca[key],bins=(x_edges),density=False)
    for j in range(len(H)):
      a = H[j]  # Maximum value
      b = bins[j]  # Peak position

      x = xArray[round(b*4-216)]

      # Calculate the y values for the curve using the Gaussian function
      y = a * np.exp(-(x - b)**2 / (2 * c**2))
      histogram, bins = np.histogram(x, bins=x_edges, density=False, weights = y)
      h_list.append(histogram)
    h_lists = sum(h_list)
    inputs.append(h_lists)
    #h_lists = []
  #print(inputs)
  H_exp, bins = np.histogram(ca_exp,bins=(x_edges),density=False)
  target = (H_exp)
  hists1 = (inputs[0])
  hists2 = (inputs[1])
  hists3 = (inputs[2])
  hists4 = (inputs[3])
  hists5 = (inputs[4])
  hists6 = (inputs[5])
  #print(np.shape(inputs),len(inputs[0]),len(target))
  data = {'x1': list(hists1), 'x2': list(hists2), 'x3': list(hists3), 'x4':list(hists4), 'x5':hists5, 'x6':list(hists6), 'y': list(target) }
  df = pd.DataFrame(data)
  x = df[['x1','x2','x3','x4','x5','x6', 'x7']]
  y = df['y']


  model = sm.GLS(y,x)
  results = model.fit()
  return results.rsquared


aDict = returnDict([-120, 150, 0, -120, -90])

%lprun -f main main(DATAINPUT, "T", aDict)

NameError: ignored

In [ ]:
#new code starts here
def returnDict(params):
  topSep = params[0]
  topH = params[1]
  middleH = params[2]
  botSep = params[3]
  botH = params[4]
  return {
      #upper left
      'rbetaA':(-180.0, topSep, topH, 180.0),
      'rppiiA':( topSep,   0.0, topH, 180.0),

      #Middle
      'rdelta':(-180.0,   0.0,  middleH, 105.0),

      #large middle box
      'rturnI':(-180.0,   0.0, botH,  middleH), # turn is wider alpha

      #smaller tiny box
      'ralpha':(-40, -35, -100, -50), # alph is strictly alpha

      #bottom left
      'rbetaB':(-180.0, botSep,-180.0, botH),
      'rppiiB':(botSep,    0.0,-180.0, botH),

      'rlhelx':(   0.0, 180.0,-180.0, 180.0),
  }
def returnDict2(params):
  topSep = params[0]
  topH = params[1]
  middleH = params[2]
  botSep = params[3]
  botH = params[4]
  boxL1 = params[5]
  boxL2 = params[6]
  boxH1 = params[7]
  boxH2 = params[8]
  return {
      #upper left
      'rbetaA':(-180.0, topSep, topH, 180.0),
      'rppiiA':( topSep,   0.0, topH, 180.0),

      #Middle
      'rdelta':(-180.0,   0.0,  middleH, 105.0),

      #large middle box
      'rturnI':(-180.0,   0.0, botH,  middleH), # turn is wider alpha

      #smaller tiny box
      'ralpha':(boxL1, boxL2, boxH1, boxH2), # alph is strictly alpha

      #bottom left
      'rbetaB':(-180.0, botSep,-180.0, botH),
      'rppiiB':(botSep,    0.0,-180.0, botH),

      'rlhelx':(   0.0, 180.0,-180.0, 180.0),

      #unknown part
      'rzetag':(-180,   0.0,   0.0,   120.0)
  }

In [ ]:
def func(params):
  #topSep, topH, middleH, botSep, botH
  theDict = returnDict(params)
  r = main(DATAINPUT, "T", theDict)
  return r * -1

In [ ]:
!pip install line_profiler
%load_ext line_profiler

TypeError: ignored

In [ ]:
#vertical divider on top (x = -180:0), height of top section (y = 0:180), middle height (y = -90ish:90ish), vertical divider on bottom (x = -180:0), height of bottom section (y = -180:0)
guess = [-90, 150, 0, -90, -150]
#bounds = ((-116, -114), (129, 131), (-21, -19), (-146, -144), (-61, -59))
result = spo.minimize(func, guess, method = 'Nelder-Mead')
#result = spo.basinhopping(func, guess)
#print(result.x)
print(f"R^2 = {result.fun * -1}")
print(returnDict(result.x))

R^2 = 0.6871396675801826
{'rbetaA': (-180.0, -79.99356945975555, 160.80518602705416, 180.0), 'rppiiA': (-79.99356945975555, 0.0, 160.80518602705416, 180.0), 'rdelta': (-180.0, 0.0, 0.00041459734287132785, 105.0), 'rturnI': (-180.0, 0.0, -170.63525106572501, 0.00041459734287132785), 'ralpha': (-40, -35, -100, -50), 'rbetaB': (-180.0, -79.19046712642192, -180.0, -170.63525106572501), 'rppiiB': (-79.19046712642192, 0.0, -180.0, -170.63525106572501), 'rlhelx': (0.0, 180.0, -180.0, 180.0)}


In [ ]:
from pandas.core.internals.construction import ma
def toTime():
  phi = DATAINPUT[0][0]
  psi = DATAINPUT[0][1]
  rchoice = returnDict([0,1,2,3,4])
  a = np.array(phi)
  b = np.array(psi)

  threshold = rchoice['rbetaA'][0]
  testmask = np.logical_and(a >= rchoice['rbetaA'][0], a <= rchoice['rbetaA'][1])
  mask1 = np.logical_and(np.array(phi) >= rchoice['rbetaA'][0], np.array(phi) <= rchoice['rbetaA'][1])
  mask1 = np.logical_and(mask1, np.array(psi) >= rchoice['rbetaA'][2])
  mask1 = np.logical_and(mask1, np.array(psi) <= rchoice['rbetaA'][3])

  mask2 = np.logical_and(np.array(phi) >= rchoice['rbetaB'][0], np.array(phi) <= rchoice['rbetaB'][1])
  mask2 = np.logical_and(mask1, np.array(psi) >= rchoice['rbetaB'][2])
  mask2 = np.logical_and(mask1, np.array(psi) <= rchoice['rbetaB'][3])

  mask = np.logical_or(mask1, mask2)
  print(len(mask))
  print(len(testmask))
%lprun -f toTime toTime()

10000
10000
